link up imgs in images/ with according id in data/legend.csv

In [1]:
import numpy as np
import data_processor

# STEP 0 - GETTING THE DATA

In [2]:
get_data_from_file = False  # faster when this is False

# get data from file
if (get_data_from_file):
    DATA_FILE = "data_images_100_100.txt"
    data = np.genfromtxt(DATA_FILE)
    X = data[:, :-1]
    y = data[:, -1]

# generate data here
if not get_data_from_file:
    DATA_IMG_SIZE = (350, 350)
    X, y = data_processor.convert_images(DATA_IMG_SIZE, True)
    X = np.array(X)
    y = np.array(y)


--- Done reading legend.csv ---
--- Done turning images into vectors ---


In [3]:
print(X.shape)
print(y.shape)

(13690, 122500)
(13690,)
